In [2]:
#### %matplotlib inline
from model import *
from agents import *
from globals import *
# from server import *
from schedule import *
from utility import *
from SALib.sample import saltelli
from mesa.batchrunner import BatchRunner, BatchRunnerMP
import pathos
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
import time
import multiprocessing as mp
import analysis



In [11]:
# We define our variables and bounds

fixed_params = {
    "network_type": 2
}

problem = {
    'num_vars':2,
    'names': ["opinions","malicious_N"],
    'bounds': [[2,5],[0,5]]
}

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 1
max_steps = 1
distinct_samples = 2

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples, calc_second_order= False)


In [ ]:
# Set the outputs
model_reporters = {
#                 "preferences": compute_preferences,
#                 "opinions": compute_opinions,
#                 "preference_A": compute_preference_A,
#                 "preference_B": compute_preference_B,
                "radical_opinions": compute_radical_opinions,
                "Silent_Spiral": compute_silent_spiral,
                "majority_percentage": compute_majority_opinions,
                "community_no": community_no,
                "transitivity": compute_transitivity,
                "echo_chamber": compute_echo_chamber,
                "echo_size": echochamber_size,
                "echo_count": echochamber_count,          
}

# param_values = DIRK
# param_values = TOBIAS

# BatchRunner
batch = BatchRunnerMP(Network, 
                    max_steps=max_steps,
                    fixed_parameters=fixed_params,
                    variable_parameters={name:[] for name in problem['names']},
                    model_reporters=model_reporters, nr_processes=4)
times = []
count = 0
for i in range(replicates):
    for vals in param_values:
        start = time.time()
        # Change parameters that should be integers
        vals = list(vals)
#         vals[2] = int(vals[2])

        # Transform to dict with parameter names and their values
        variable_parameters = {}
        for name, val in zip(problem['names'], vals):
            variable_parameters[name] = val
        variable_parameters['network_type']=2
        variable_parameters['malicious_N']=int(variable_parameters['malicious_N'])
        variable_parameters['N']=1000
        variable_parameters['beta_component']=1
        variable_parameters['swingers']=3
        variable_parameters['echo_limit']=0.95
        variable_parameters['social_influence']=0.06
        variable_parameters['similarity_treshold']=0.16
        variable_parameters['no_of_neighbors']=3
        variable_parameters['all_majority']=False
        variable_parameters['opinions']=int(variable_parameters['opinions'])

        batch.run_iteration(variable_parameters, tuple(vals), count)
        count += 1

        clear_output()
        print(f'{count / (len(param_values) * (replicates)) * 100:.2f}% done')
        times.append(time.time() - start)
        meantime = np.mean(times)
        print('Average duration per iteration: %s seconds.'%str(meantime))
        nectime = ((len(param_values) * (replicates))-count)*meantime
        print('Expected time till finish: ', int(nectime/3600), 'hours and ', int((nectime%3600)/60), ' minutes.')
        
        
    
data = batch.get_model_vars_dataframe()
data.to_pickle('./Data/Majority_Minority.pkl')


12.50% done
Average duration per iteration: 6.75390625 seconds.
Expected time till finish:  0 hours and  0  minutes.
